<a href="https://colab.research.google.com/github/drhdt-raipur/ai_for_dld_udemy/blob/main/convert_txt_to_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from IPython.display import HTML, display
import nbformat as nbf
import uuid
from google.colab import files

# Step 1: Show HTML UI
html_ui = """
<textarea id="markdownInput" style="width:100%;height:300px;" placeholder="Paste your markdown here..."></textarea><br>
<button onclick="google.colab.kernel.invokeFunction('notebook.convert_markdown_to_ipynb', [document.getElementById('markdownInput').value], {})">
Generate Notebook and Download
</button>
"""
display(HTML(html_ui))

# Step 2: Define backend function to convert and download notebook
def convert_markdown_to_ipynb(markdown_text):
    lines = markdown_text.split('\n')
    cells = []
    current_cell_lines = []
    cell_type = 'markdown'

    for line in lines:
        if line.strip().startswith("```"):
            if current_cell_lines:
                cells.append(
                    nbf.v4.new_markdown_cell('\n'.join(current_cell_lines)) if cell_type == 'markdown'
                    else nbf.v4.new_code_cell('\n'.join(current_cell_lines))
                )
                current_cell_lines = []
            cell_type = 'code' if cell_type == 'markdown' else 'markdown'
        else:
            current_cell_lines.append(line)

    if current_cell_lines:
        cells.append(
            nbf.v4.new_markdown_cell('\n'.join(current_cell_lines)) if cell_type == 'markdown'
            else nbf.v4.new_code_cell('\n'.join(current_cell_lines))
        )

    # Create notebook object
    nb = nbf.v4.new_notebook()
    nb['cells'] = cells
    nb['metadata'] = {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "name": "python",
            "version": "3.10"
        }
    }

    # Save notebook to a Colab-downloadable location
    filename = f"generated_{uuid.uuid4().hex[:6]}.ipynb"
    with open(filename, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

    # Trigger browser download
    files.download(filename)

# Step 3: Register callback
from google.colab import output
output.register_callback('notebook.convert_markdown_to_ipynb', convert_markdown_to_ipynb)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>